In [1]:
# Wikipedia API Retriever Tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wikipedia_retriever_tool = WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)

In [2]:
# Web page Retriever Tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool

loader = WebBaseLoader("https://docs.smith.langchain.com/")
web_docs = loader.load()
web_documents = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200).split_documents(web_docs)
web_vector_db = FAISS.from_documents(web_documents, OllamaEmbeddings(model="llama3.2"))
web_retriever = web_vector_db.as_retriever()


web_retriever_tool = create_retriever_tool(retriever=web_retriever, 
                                           name="langsmith_search", 
                                           description="Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# ArXiv Retriever Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_retriever_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [4]:
tools = [wikipedia_retriever_tool, arxiv_retriever_tool, web_retriever_tool]

In [5]:
import os
from dotenv import load_dotenv
from langchain_ollama.chat_models import ChatOllama
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="false"

llm = ChatOllama(model="llama3.2")


In [14]:
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor

prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

5. Run your first evaluation​
Evaluation requires a system to test, data to serve as test cases, and optionally evaluators to grade the results. Here we use a built-in accuracy evaluator.

View a sample output trace.
Learn more about tracing in the how-to guides.

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Cloud)AdministrationSelf-hostingReferenceQuick startOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!
1. Install LangSmith​
PythonTypeScriptpip install -U langsmithyarn add langsmith
2. Create an API key​
To create an API

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\n\nTo get started with LangSmith, follow these steps:\n\n1. Install LangSmith using pip or yarn.\n2. Create an API key by heading to the Settings page and clicking "Create API Key".\n3. Set up your environment by exporting the necessary variables, including `LANGCHAIN_TRACING_V2` and `LANGCHAIN_API_KEY`.\n4. Log your first trace using the provided tools and guides.\n\nLangSmith provides multiple ways to log traces, including using the `traceable` library. You can also use tracing for LangChain users without directly using the LangSmith SDK.\n\nOverall, LangSmith is a powerful tool for building and evaluating LLM applications, and its simple setup and usage make it accessible to developers of all levels.'}

In [22]:
agent_executor.invoke({"input":"Who is the author of the paper in ArXiv titled MAMI: Multi-Attentional Mutual-Information for Long Sequence Neuron Captioning ?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'MAMI: Multi-Attentional Mutual-Information for Long Sequence Neuron Captioning author'}`


Published: 2024-01-05
Title: MAMI: Multi-Attentional Mutual-Information for Long Sequence Neuron Captioning
Authors: Alfirsa Damasyifa Fauzulhaq, Wahyu Parwitayasa, Joseph Ananda Sugihdharma, M. FadliThe authors of the paper in ArXiv titled "MAMI: Multi-Attentional Mutual-Information for Long Sequence Neuron Captioning" are:

1. Alfirsa Damasyifa Fauzulhaq
2. Wahyu Parwitayasa
3. Joseph Ananda Sugihdharma
4. M. Fadli

> Finished chain.


{'input': 'Who is the author of the paper in ArXiv titled MAMI: Multi-Attentional Mutual-Information for Long Sequence Neuron Captioning ?',
 'output': 'The authors of the paper in ArXiv titled "MAMI: Multi-Attentional Mutual-Information for Long Sequence Neuron Captioning" are:\n\n1. Alfirsa Damasyifa Fauzulhaq\n2. Wahyu Parwitayasa\n3. Joseph Ananda Sugihdharma\n4. M. Fadli'}

In [12]:
# Using ReAct Agent because the openai agent doesn't support Ollama
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.tools import render_text_description

prompt = hub.pull("hwchase17/react-json")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, format="json", handle_parsing_errors=True, verbose=True)

agent_executor.invoke({"input": "tell me about LangSmith"})



> Entering new AgentExecutor chain...
Thought: I'm not familiar with a person or entity named LangSmith, but I'll try to find some information.
Action:
```
{
  "action": "wikipedia",
  "action_input": "LangSmith"
}
``Invalid Format: Missing 'Action Input:' after 'Action:'Could not parse LLM output: `Thought: I'll add the missing 'Action Input:' after 'Action:' to complete the JSON blob.
Action:
```
{
  "action": "wikipedia",
  "action_input": "LangSmith`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILUREInvalid or incomplete responseThought: I'll try rephrasing the search query.
Action:
```
{
  "action": "wikipedia",
  "action_input": "LangSmith"
}
``Invalid Format: Missing 'Action Input:' after 'Action:'Thought: I'll rephrase the search query again to include the correct format for the Wikipedia action.
Action:
```
{
  "action": "wikipedia",
  "action_input": "LangSmithInvalid Format: Missing 'Action Input:' after 'Action:'Tho

{'input': 'tell me about LangSmith',
 'output': 'Langsmith is not a well-known person or entity, but I found that Langford Smith was a British engineer and inventor.'}

In [11]:
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(
    instructions=instructions,
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools])
    )
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True)

agent_executor.invoke({"input": "tell me about LangSmith"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: tell me about LangSmithPage: Which Brings Me to You
Summary: Which Brings Me to You is a 2024 American romantic comedy film directed by Peter Hutchings and starring Lucy Hale and Nat Wolff. It is an adaptation by Keith BuniThought: I've obtained information about LangSmith from Wikipedia.

Final Answer: Langsmith is not a well-known entity in my current knowledge base. The previous conversation mentioned LangSmith, which brought the user to the movie "Which Brings Me to You". However, it's unclear what LangSmith refers to specifically. Can you please provide more context or clarify what LangSmith represents?

> Finished chain.


{'input': 'tell me about LangSmith',
 'output': 'Langsmith is not a well-known entity in my current knowledge base. The previous conversation mentioned LangSmith, which brought the user to the movie "Which Brings Me to You". However, it\'s unclear what LangSmith refers to specifically. Can you please provide more context or clarify what LangSmith represents?'}